# Lab Exercice 02

## Instructions

The csv files are a collection of tweets labelled with sentiment in 3 categories:

sentiments = {
    "LABEL_0": "Bearish", 
    "LABEL_1": "Bullish", 
    "LABEL_2": "Neutral"
}  

Train a LSTM network to with the training file. Validate the trained model with the valid file. Comment what you are doing in each part of your code. As the better the code, comments and result validation as the better the grade.